<a href="https://colab.research.google.com/github/0906Bao/TriTueNhanTao/blob/main/Tuan5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.datasets import make_blobs
import gradio as gr

In [2]:
def generate_data(n_samples=300, n_features=2, centers=3, random_state=0):
    X, y = make_blobs(
        n_samples=n_samples,
        n_features=n_features,
        centers=centers,
        cluster_std=1.2,
        random_state=random_state
    )
    return X, y

Hàm sinh dữ liệu

Thực hiện các việc:
* Khởi tạo tập dữ liệu nhân tạo hai chiều bằng phương pháp sinh cụm.
* Xác định số lượng mẫu, số đặc trưng, số cụm và độ phân tán.
* Sinh các điểm dữ liệu xoay quanh các tâm cụm theo phân phối Gaussian.
* Gán nhãn tương ứng cho mỗi điểm dữ liệu.
* Trả về tập đặc trưng và tập nhãn dùng cho các thuật toán học máy.



In [3]:
def kmeans_init_centers(X, k):
    return X[np.random.choice(X.shape[0], k, replace=False)]


def kmeans_assign_labels(X, centers):
    distances = cdist(X, centers)
    return np.argmin(distances, axis=1)


def kmeans_update_centers(X, labels, k):
    centers = np.zeros((k, X.shape[1]))
    for i in range(k):
        centers[i] = np.mean(X[labels == i], axis=0)
    return centers


def kmeans(X, k, max_iter=50):
    centers = kmeans_init_centers(X, k)
    for _ in range(max_iter):
        labels = kmeans_assign_labels(X, centers)
        new_centers = kmeans_update_centers(X, labels, k)
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return labels, centers

**Hàm kmeans_init_centers**

Thực hiện các việc:
*  Nhận vào tập dữ liệu và số lượng cụm cần phân chia.
*  Chọn ngẫu nhiên một số điểm dữ liệu bằng với số cụm.
*  Sử dụng các điểm được chọn làm tâm cụm ban đầu.
*  Trả về danh sách tâm cụm khởi tạo.

**Hàm kmeans_assign_labels**

Thực hiện các việc:
*  Tính khoảng cách từ mỗi điểm dữ liệu đến tất cả các tâm cụm.
*  Lưu trữ khoảng cách dưới dạng ma trận.
*  Xác định tâm cụm gần nhất đối với từng điểm dữ liệu.
*  Gán nhãn cụm cho từng điểm dựa trên khoảng cách nhỏ nhất.
*  Trả về mảng nhãn của toàn bộ dữ liệu

**Hàm kmeans_update_centers**

Thực hiện các việc:

*  Khởi tạo mảng rỗng để lưu các tâm cụm mới.
*  Duyệt qua từng cụm dữ liệu.
*  Lọc các điểm thuộc cùng một cụm.
*  Tính trung bình tọa độ các điểm trong cụm.
*  Gán giá trị trung bình làm tâm cụm mới.
*  Trả về danh sách tâm cụm sau khi cập nhật.

**Hàm kmeans**

Thực hiện các việc:

*  Khởi tạo tâm cụm ban đầu.
*  Lặp quá trình phân cụm theo số vòng lặp cho phép.
*  Gán nhãn cụm cho toàn bộ dữ liệu.
*  Cập nhật lại vị trí tâm cụm.
*  So sánh tâm cũ và tâm mới để kiểm tra hội tụ.
*  Kết thúc thuật toán khi tâm cụm không còn thay đổi đáng kể.
*  Trả về nhãn phân cụm và vị trí tâm cuối cùng.

In [4]:
def knn_predict(X_train, y_train, X_test, k):
    distances = np.sqrt(
        np.sum((X_test[:, None, :] - X_train[None, :, :]) ** 2, axis=2)
    )
    predictions = []
    for i in range(X_test.shape[0]):
        idx = np.argsort(distances[i])[:k]
        labels, counts = np.unique(y_train[idx], return_counts=True)
        predictions.append(labels[np.argmax(counts)])
    return np.array(predictions)

Hàm phân lớp K-Nearest Neighbors

Thực hiện các việc:

* Nhận vào dữ liệu huấn luyện, nhãn huấn luyện, dữ liệu cần dự đoán và tham số k.
* Tính khoảng cách Euclid giữa từng điểm kiểm tra và toàn bộ dữ liệu huấn luyện.
* Sắp xếp các khoảng cách theo thứ tự tăng dần.
* Lấy k điểm huấn luyện gần nhất.
* Trích xuất nhãn của các láng giềng gần nhất.
* Thực hiện bỏ phiếu đa số để xác định nhãn dự đoán.
* Trả về danh sách nhãn dự đoán cho các điểm kiểm tra.

In [5]:
def plot_clusters(X, labels, centers=None, title=""):
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.scatter(X[:, 0], X[:, 1], c=labels, s=30)
    if centers is not None:
        ax.scatter(centers[:, 0], centers[:, 1], c='red', s=200, marker='X')
    ax.set_title(title)
    return fig

Hàm vẽ đồ thị phân cụm / phân lớp

Thực hiện các việc:

* Tạo đối tượng đồ thị bằng thư viện Matplotlib.
* Vẽ các điểm dữ liệu trên mặt phẳng hai chiều.
* Gán màu cho từng điểm dựa trên nhãn cụm hoặc nhãn lớp.
* Nếu có tâm cụm, vẽ tâm bằng ký hiệu riêng biệt.
* Gán tiêu đề cho đồ thị.
* Trả về đối tượng đồ thị để hiển thị trên giao diện Gradio.

In [6]:
X_global, y_global = generate_data()


def run_clustering(algorithm, k):
    if algorithm == "K-Means":
        labels, centers = kmeans(X_global, k)
        fig = plot_clusters(X_global, labels, centers, "K-Means Clustering")
    else:
        labels = knn_predict(X_global, y_global, X_global, k)
        fig = plot_clusters(X_global, labels, None, "KNN Classification")
    return fig


Hàm điều phối thuật toán theo lựa chọn người dùng

Thực hiện các việc:

* Nhận thông tin thuật toán được lựa chọn và giá trị k.
* Kiểm tra loại thuật toán được chọn.
* Nếu là K-Means, thực hiện phân cụm dữ liệu và lấy tâm cụm.
* Nếu là KNN, thực hiện phân lớp dữ liệu dựa trên nhãn có sẵn.
* Gọi hàm vẽ đồ thị để trực quan hóa kết quả.
* Trả về biểu đồ kết quả cho giao diện Gradio.

In [7]:
with gr.Blocks(title="Clustering Demo: K-Means & KNN") as demo:
    gr.Markdown("# Demo phân cụm & phân lớp: K-Means và KNN")

    with gr.Row():
        algorithm = gr.Radio([
            "K-Means",
            "KNN"
        ], label="Chọn thuật toán", value="K-Means")

        k_value = gr.Slider(
            minimum=1,
            maximum=10,
            step=1,
            value=3,
            label="Giá trị k"
        )

    run_button = gr.Button("Chạy thuật toán")
    output_plot = gr.Plot()

    run_button.click(
        fn=run_clustering,
        inputs=[algorithm, k_value],
        outputs=output_plot
    )

In [8]:
if __name__ == '__main__':
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://20d84d8d6743b6c349.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
